In [ ]:
import os
from google.colab import drive

drive.mount("/content/drive")
os.chdir("/content/drive/My Drive/Colab Notebooks/CRC")

Mounted at /content/drive


In [ ]:
!pip install networkit

     |████████████████████████████████| 3.1MB 24.6MB/s 
  Created wheel for networkit: filename=networkit-7.1-cp36-cp36m-linux_x86_64.whl size=8053156 sha256=d34de80d504f41a4729e2122afeb1ed53e60de402cc7939fc02f7606fd4be9fd
  Stored in directory: /root/.cache/pip/wheels/a2/ca/1c/a1096b1823f845c29784ce5106866b01eaad26d101a79032b8
Successfully built networkit


In [ ]:
!bash pytorch041_cuda92_colab.sh

--2020-12-08 23:21:16--  https://developer.nvidia.com/compute/cuda/9.2/Prod2/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod2/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64.deb?qLpeSqN-UGS4IhwReBo5VXXRj8aYZazAD15Pg8FRzgYKcj4HtG3ARcDaRnBAmdBiRI2HADszCQMP1HqfO3fFq1-gd93vuAQh71w_4kw9uMuxRUzPfu8YXyv9kZcqkT5H3sufYpwNJ4Eb7tnGu2nCuNfGf75-qGyxPc3SUF2K0OQ-dCTY2tUn1O4ceeJEk0S7NDjR-rOCXVgmXxdoNjwkuA [following]
--2020-12-08 23:21:16--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod2/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64.deb?qLpeSqN-UGS4IhwReBo5VXXRj8aYZazAD15Pg8FRzgYKcj4HtG3ARcDaRnBAmdBiRI2HADszCQMP1HqfO3fFq1-gd93vuAQh71w_4kw9uMuxRUzPfu8

In [ ]:
import time
import torch
import pickle
import random
import numpy as np
import networkx as nx
import torch.nn as nn

from model import GNN_Bet
from utils import *

torch.manual_seed(20)

#Model parameters
model_size = 10000
hidden = 20
num_epoch = 10


print(f"Loading data...")
with open("./training.pickle", "rb") as fopen:
    list_graph_train, list_n_seq_train, list_num_node_train, bc_mat_train = pickle.load(fopen)


with open("./test.pickle", "rb") as fopen:
    list_graph_test, list_n_seq_test, list_num_node_test, bc_mat_test = pickle.load(fopen)


#Get adjacency matrices from graphs
print(f"Graphs to adjacency conversion.\n")
list_adj_train, list_adj_t_train = graph_to_adj(
    list_graph_train,
    list_n_seq_train,
    list_num_node_train,
    model_size
)
list_adj_test, list_adj_t_test = graph_to_adj(
    list_graph_test,
    list_n_seq_test,
    list_num_node_test,
    model_size
)


def train(list_adj_train, list_adj_t_train, list_num_node_train, bc_mat_train):
    model.train()
    loss_train = 0
    num_samples_train = len(list_adj_train)

    for i in range(num_samples_train):
        adj = list_adj_train[i]
        adj_t = list_adj_t_train[i]

        adj = adj.to(device)
        adj_t = adj_t.to(device)

        num_nodes = list_num_node_train[i]

        optimizer.zero_grad()

        y_out = model(adj, adj_t)
        true_arr = torch.from_numpy(bc_mat_train[:, i]).float()
        true_val = true_arr.to(device)

        loss_rank = loss_cal(y_out, true_val, num_nodes, device, model_size)
        loss_train = loss_train + float(loss_rank)
        loss_rank.backward()
        optimizer.step()


def test(list_adj_test, list_adj_t_test, list_num_node_test, bc_mat_test):
    model.eval()
    list_kt = list()
    num_samples_test = len(list_adj_test)

    for j in range(num_samples_test):
        adj = list_adj_test[j]
        adj_t = list_adj_t_test[j]

        adj = adj.to(device)
        adj_t = adj_t.to(device)

        num_nodes = list_num_node_test[j]

        y_out = model(adj, adj_t)

        print(f"Predicted top 10: {np.argsort(y_out.cpu().numpy()[:, 0])[:10]}")
        print(f"Real top 10:      {np.argsort(bc_mat_test[:, j])[:10]}")

        true_arr = torch.from_numpy(bc_mat_test[:, j]).float()
        true_val = true_arr.to(device)

        kt = ranking_correlation(y_out, true_val, num_nodes, model_size)
        list_kt.append(kt)
        print(f"Kendall's tau: {kt}")

    print(f"Average kendall tau score is: {np.mean(np.array(list_kt))} and std: {np.std(np.array(list_kt))}")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GNN_Bet(ninput=model_size, nhid=hidden, dropout=0.6)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


epoch_durations = []
print(f"Number of epochs: {num_epoch}")
for e in range(num_epoch):
    print(f"Epoch number: {e}")
    start_time = time.process_time()
    train(list_adj_train, list_adj_t_train, list_num_node_train, bc_mat_train)
    duration = time.process_time() - start_time
    epoch_durations.append(duration)
    print(f"Duration: {duration} seconds")

print()
print(f"Mean training time per epoch: {np.mean(np.array(epoch_durations))} seconds")
print(f"Total training time over {num_epoch} epochs: {np.sum(np.array(epoch_durations))} seconds")

Loading data...
Graphs to adjacency conversion.

Number of epochs: 10
Epoch number: 0
Duration: 8.383447691 seconds
Epoch number: 1
Duration: 8.148281421000007 seconds
Epoch number: 2
Duration: 8.17765569400001 seconds
Epoch number: 3
Duration: 8.067843786999987 seconds
Epoch number: 4
Duration: 8.27866935099999 seconds
Epoch number: 5
Duration: 8.252897417 seconds
Epoch number: 6
Duration: 8.173521614999999 seconds
Epoch number: 7
Duration: 8.223331607000006 seconds
Epoch number: 8
Duration: 8.244271018999996 seconds
Epoch number: 9
Duration: 8.231173829 seconds

Mean training time per epoch: 8.2181093431 seconds
Total training time over 10 epochs: 82.181093431 seconds


In [ ]:
def test(list_adj_test, list_adj_t_test, list_num_node_test, bc_mat_test):
    model.eval()
    list_kt = list()
    num_samples_test = len(list_adj_test)

    for j in range(num_samples_test):
        adj = list_adj_test[j]
        adj_t = list_adj_t_test[j]

        adj = adj.to(device)
        adj_t = adj_t.to(device)

        num_nodes = list_num_node_test[j]

        y_out = model(adj, adj_t)

        print(f"Predicted top 10: {np.argsort(y_out.cpu().numpy()[:, 0])[:10]}")
        print(f"Real top 10:      {np.argsort(bc_mat_test[:, j])[:10]}")

        true_arr = torch.from_numpy(bc_mat_test[:, j]).float()
        true_val = true_arr.to(device)

        kt = ranking_correlation(y_out, true_val, num_nodes, model_size)
        list_kt.append(kt)
        print(f"Kendall's tau: {kt}")

    print(f"Average kendall tau score is: {np.mean(np.array(list_kt))} and std: {np.std(np.array(list_kt))}")

#test on 10 test graphs and print average KT Score and its stanard deviation
with torch.no_grad():
    test(list_adj_test, list_adj_t_test, list_num_node_test, bc_mat_test)

Predicted top 10: [   0 6475 6476 6477 6478 6479 6480 6473 6481 6483]
Real top 10:      [   0 6475 6476 6477 6478 6479 6480 6473 6481 6483]
Kendall's tau: 0.9762643133991888
Predicted top 10: [   0 6710 6711 6712 6713 6714 6715 6709 6716 6718]
Real top 10:      [   0 6710 6711 6712 6713 6714 6715 6709 6716 6718]
Kendall's tau: 0.9726624872727427
Predicted top 10: [   0 6663 6664 6665 6666 6667 6668 6669 6670 6671]
Real top 10:      [   0 6663 6664 6665 6666 6667 6668 6669 6670 6671]
Kendall's tau: 0.971690303524069
Predicted top 10: [4999 6490 6491 6493 6494 6495 6496 6489 6497 6500]
Real top 10:      [4999 6490 6491 6493 6494 6495 6496 6489 6497 6500]
Kendall's tau: 0.9781816675545318
Predicted top 10: [4999 6663 6664 6665 6666 6667 6668 6669 6670 6671]
Real top 10:      [4999 6663 6664 6665 6666 6667 6668 6669 6670 6671]
Kendall's tau: 0.9776804549816458
Predicted top 10: [   0 6591 6592 6593 6594 6595 6596 6590 6597 6599]
Real top 10:      [   0 6591 6592 6593 6594 6595 6596 6590 65

In [ ]:
def test_graph(file):
    if file[-3:] == "csv":
        sep = ','
    else:
        sep = None

    G = nx.DiGraph()
    aux = {}

    print(f"Reading file {file}")
    with open(file, "r", encoding="utf-8") as graph:
        lines = graph.readlines()

        for i, line in enumerate(lines):
            edge = [int(i) for i in line.split(sep)]

            for i, e in enumerate(edge):
                if e not in aux:
                    aux[e] = len(aux)

                edge[i] = aux[e]

            G.add_edge(edge[0], edge[1])

    print(f"Nodes: {len(G.nodes)}")
    print(f"Edges: {len(G.edges)}")

    print("Computing real betweenness centrality")
    start_time = time.process_time()
    G_bc = nx.algorithms.centrality.betweenness_centrality(G)
    end_time = time.process_time() - start_time
    print(f"Brandes' algorithm duration: {end_time} seconds")

    print("Preparing inputs")
    G_list = [G]
    G_node_seq = [list(G.nodes)]
    G_num_node = [len(G.nodes)]
    G_bc_mat = np.zeros((model_size, 1))

    for i, node in enumerate(G_node_seq[0]):
        G_bc_mat[i, 0] = G_bc[node]

    G_list_adj, G_list_adj_t = graph_to_adj(
        G_list,
        G_node_seq,
        G_num_node,
        model_size
    )

    print(f"Testing model on {file} graph")
    start_time = time.process_time()
    with torch.no_grad():
        test(G_list_adj, G_list_adj_t, G_num_node, G_bc_mat)
    end_time = time.process_time() - start_time
    print(f"Model running duration: {end_time} seconds\n")


graphs = ["email.txt", "twitch.csv", "Wiki-Vote.txt"]

for graph in graphs:
    test_graph(graph)

Reading file email.txt
Nodes: 1005
Edges: 25571
Computing real betweenness centrality
Brandes' algorithm duration: 4.8110491489999845 seconds
Preparing inputs
Testing model on email.txt graph
Predicted top 10: [ 160   62   86  107  434  121  183 6668 6669 6670]
Real top 10:      [4999 6663 6664 6665 6666 6667 6668 6669 6670 6671]
Kendall's tau: 0.04069683343291092
Average kendall tau score is: 0.04069683343291092 and std: 0.0
Model running duration: 0.01792966499999693 seconds

Reading file twitch.csv
Nodes: 9498
Edges: 153138
Computing real betweenness centrality
Brandes' algorithm duration: 291.062381285 seconds
Preparing inputs
Testing model on twitch.csv graph
Predicted top 10: [  71  458  834   44   54   41 1293   57    4  266]
Real top 10:      [   0 7069 7068 7066 7065 7063 7053 7074 7052 7048]
Kendall's tau: 0.4727833848948767
Average kendall tau score is: 0.4727833848948767 and std: 0.0
Model running duration: 0.053059887000017625 seconds

Reading file Wiki-Vote.txt
Nodes: 711